## Table of Contents
- [Random Forest Regression](#Random-Forest-Regression)
  - [Concept](#Concept)
  - [How It Works](#How-It-Works)
  - [Key Parameters](#Key-Parameters)
  - [Advantages](#Advantages)
  - [Applications](#Applications)
- [Implementation](#Implementation)
  - [Parameters](#Parameters)
- [🏠 Home](../../../../welcomePage.ipynb)

# Random Forest Regression

**Random Forest Regression** is an ensemble learning method for regression that operates by constructing a multitude of decision trees at training time and outputting the mean prediction of the individual trees. Random Forest combines multiple decision trees to increase the overall accuracy and reduce the risk of overfitting.

## Concept

Random Forest builds multiple decision trees and merges them together to get a more accurate and stable prediction. The ensemble of trees helps to correct for trees that may have overfitted to their sample. It is one of the most powerful and widely used machine learning algorithms due to its simplicity and diversity (applicable to both classification and regression tasks).

## How It Works

1. **Bootstrap Aggregating (Bagging):** Random Forest uses the bagging technique where it creates multiple datasets from the original dataset by sampling with replacement, and then trains a decision tree on each.
2. **Node Splitting:** At each node in the trees, a random subset of the features is considered for splitting. This introduces diversity in the model, reducing the variance and preventing overfitting.
3. **Averaging:** For regression tasks, it averages the outputs of different trees to give a final prediction.

## Key Parameters

- $n_{\text{estimators}}$: The number of trees in the forest. Typically, the more trees, the better to learn the data. However, adding a lot of trees can slow down the training process considerably.
- $\text{max\_depth}$: The maximum depth of the trees. Deeper trees learn more detailed data patterns and can be more prone to overfitting.
- $\text{min\_samples\_split}$: The minimum number of samples required to split an internal node.
- $\text{min\_samples\_leaf}$: The minimum number of samples required to be at a leaf node.

## Advantages

- Effective in high dimensional spaces and large data sets.
- Handles both numerical and categorical data.
- Provides a good indicator of the importance of features.

## Applications

Random Forest is versatile and can be used for:
- Predicting housing prices,
- Stock market behavior analysis,
- Medical diagnosis,
- Complex regression tasks in ecological and biological studies.

# Implementation

### Import Libraries

**Press ▶ to import the libraries.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from IPython.display import display, clear_output, HTML

import warnings
warnings.filterwarnings("ignore")

print("Libraries are imported.")

### Import and show Data

**Press ▶ to load the data.**

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# List all .csv and Excel files in the current directory
supported_extensions = ['.csv', '.xlsx', '.xls']
files = [f for f in os.listdir('./Data') if any(f.endswith(ext) for ext in supported_extensions)]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=files,
    description='Files:',
    disabled=False,
)

# Create a button widget
button = widgets.Button(
    description='Select',
    disabled=False,
    button_style='',
    tooltip='Click to select file',
    icon='check'
)

# Output widget to display messages
output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    with output:
        clear_output()
        selected_file = dropdown.value
        global data
        if selected_file.endswith('.csv'):
            data = pd.read_csv("./Data/" +selected_file)
        elif selected_file.endswith(('.xlsx', '.xls')):
            data = pd.read_excel("./Data/" +selected_file)
        print(f"File '{selected_file}' uploaded as data.")

# Attach the function to the button widget
button.on_click(on_button_click)

# Display the dropdown, button widgets, and initial message within the output widget
with output:
    print("Please select a file from the dropdown and click 'Select'.")
display(output)
display(dropdown)
display(button)


**Press ▶ to display the data.**

In [ ]:
display(data.head())
print ("The data is composed of ", data.shape[0], " rows and ", data.shape[1], " columns.")

### Data Preprocessing

**Press ▶ to specify the target column.**

In [ ]:
import ipywidgets as widgets
import pandas as pd

# Create a Dropdown widget for column selection
dropdown = widgets.Dropdown(
    options=data.columns.tolist(),
    value=data.columns[0],
    description='Select Target Column:',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

# Create a Button widget
button = widgets.Button(
    description='Select',
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to select the target column as the last column',
    icon='check'  # FontAwesome icon names (without 'fa-')
)

# Create an Output widget for displaying messages
output = widgets.Output()

# Function to handle button click that rearranges the DataFrame
def on_button_clicked(b):
    with output:
        output.clear_output()
        global data
        # Get the selected column name
        selected_column = dropdown.value
        # Reorder the DataFrame columns
        new_columns = [col for col in data.columns if col != selected_column] + [selected_column]
        data = data[new_columns]
        print(f"Column '{selected_column}' has been moved to the last position.")

# Link the button click event to the function
button.on_click(on_button_clicked)

# Display the widgets and output
display(widgets.VBox([dropdown, button, output]))


**Press ▶ to create a lagged target column.**

In [ ]:
target = data.columns[-1]
data['Target_Lag1'] = data.iloc[:, -1].shift(1)

data.dropna(inplace=True)

### Predict Bead Area

## Parameters

### Number of Estimators
Number of estimators refers to the number of decision trees in the forest. This parameter controls how many trees are built in the model. More trees generally improve performance and model stability by reducing overfitting but increase computational cost. Properly setting the number of estimators balances accuracy and computational efficiency.

### Maximum Depth
Maximum depth is the maximum number of levels each tree in the forest can have. This parameter controls the depth of the trees. Deeper trees can capture more detailed patterns but also increase the risk of overfitting. Limiting the maximum depth helps prevent overfitting while capturing essential interactions in the data.

### Minimum Split Samples
Minimum split samples refer to the minimum number of samples required to split an internal node. This parameter determines the smallest amount of data a node must have to consider splitting. Higher values prevent the model from learning overly specific patterns (overfitting) by ensuring splits are made only when there is sufficient data. Properly tuning this parameter helps maintain a balance between model complexity and generalization.


**Press ▶ to specify independent variables, train/test split, and the model parameter and to forecast the data.**

In [ ]:


# Define widgets with adjusted layout
index_range_slider = widgets.IntRangeSlider(
    value=[0, min(500, len(data))],
    min=0,
    max=len(data),
    step=1,
    description='Index Range:',
    layout=widgets.Layout(width='600px'),  # Increase width for better readability
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

feature_select = widgets.SelectMultiple(
    options=tuple(col for col in data.columns if col != target),
    value=tuple(col for col in data.columns if col != target),
    description='Features:',
    layout=widgets.Layout(width='600px', height='180px'),  # Increase width and height
    style={'description_width': '150px'},  # Increase description width
    disabled=False
)

train_size_slider = widgets.IntSlider(
    value=80,
    min=50,
    max=95,
    step=1,
    description='Train %:',
    layout=widgets.Layout(width='600px'),  # Increase width
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

# Random Forest parameter sliders
n_estimators_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Number of Estimators:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '200px'},
    continuous_update=False
)

max_depth_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='Maximum Depth:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '200px'},
    continuous_update=False
)

min_samples_split_slider = widgets.IntSlider(
    value=2,
    min=2,
    max=20,
    step=1,
    description='Minimum Split Samples:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '200px'},
    continuous_update=False
)

apply_button = widgets.Button(description="Apply Changes", layout=widgets.Layout(width='800px'))

# Define the function to apply changes and update the plots
def apply_changes(b):
    with output:
        clear_output(wait=True)
        
        # Extract the parameters from widgets
        index_range = index_range_slider.value
        selected_features = list(feature_select.value)
        train_size_pct = train_size_slider.value / 100
        n_estimators = n_estimators_slider.value
        max_depth = max_depth_slider.value
        min_samples_split = min_samples_split_slider.value
        
        # Slice the data
        df = data[index_range[0]:index_range[1]]
        
        # Prepare the data (assuming 'Interpolated Bead Area' is already in `df`)
        X = df[selected_features]
        y = df[target]
        
        # Train-test split
        train_size = int(len(df) * train_size_pct)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Train the model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=42
        )
        model.fit(X_train, y_train)
        
        # Predict on test data
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        display(HTML(f'<b>Mean Squared Error: {mse:.5f}</b>'))  # Display MSE in bold
        
        # Plot predicted vs actual
        plt.figure(figsize=(10, 6))
        plt.plot(y_train.index, y_train, label='Training', color='green')
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel(target)
        plt.title('Actual vs Predicted '+target)
        plt.legend()
        plt.show()
        
        # Calculate loss for each point
        pointwise_mse_loss = (y_test - y_pred) ** 2
        
        # Plot the pointwise loss
        plt.figure(figsize=(10, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.plot(y_test.index, pointwise_mse_loss, label='Pointwise MSE Loss', color='orange')
        plt.xlabel('Time')
        plt.ylabel('MSE Loss')
        plt.title('Pointwise MSE Loss of Predicted vs Actual '+target)
        plt.legend()
        plt.show()

# Link the apply button to the function
apply_button.on_click(apply_changes)

# Display the widgets and the output area
output = widgets.Output()

display(index_range_slider, feature_select, train_size_slider, n_estimators_slider, max_depth_slider, min_samples_split_slider, apply_button, output)


### <center>[🏠 Home](../../../../welcomePage.ipynb)</center>